In [7]:
import requests
import pandas as pd

from PIL import Image
from io import BytesIO
import pytesseract

In [8]:
## Fetch an OAUTH Token. It is only valid for 2 hours and will need to be re-requested when it expires.

# The following code curtesy of James Briggs: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('fOIoNDAQnOFBiQ5RM9CEuA', 'Rb9whMn_xOwNVra9usbbEPYkVtMqvw')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'flatest_iron',
        'password': 'FlatironPassword19!'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [9]:
# This is just a test to see how this works in general

# Get the top 25 posts of all time from r/python
#TODO: It might be worth it to change which posts are pulled: maybe hot posts are more indicative, but for now we're doing this
response = requests.get("https://oauth.reddit.com/r/python/top/?t=all", headers=headers)

# python_df = pd.DataFrame()

# #We can add to this if need be; there is a lot of data that isn't being captured here
# for post in response.json()['data']['children']:
#     python_df = python_df.append({
#         'subreddit' : post['data']['subreddit'],
#         'title' : post['data']['title'],
#         'text' : post['data']['selftext'],
#         'upvote_ratio' : post['data']['upvote_ratio'],
#         'url_overriden_by_dest' : post['data']['url_overridden_by_dest'] #I think this is the image url
#     }, ignore_index = True)

# python_df.head()

In [10]:
response.json()['data']['children'][0]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'Python',
  'selftext': '',
  'author_fullname': 't2_g532u',
  'saved': False,
  'mod_reason_title': None,
  'gilded': 0,
  'clicked': False,
  'title': "Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.",
  'link_flair_richtext': [{'e': 'text', 't': 'Discussion'}],
  'subreddit_name_prefixed': 'r/Python',
  'hidden': False,
  'pwls': 6,
  'link_flair_css_class': 'discussion',
  'downs': 0,
  'thumbnail_height': 68,
  'top_awarded_type': None,
  'hide_score': False,
  'name': 't3_g53lxf',
  'quarantine': False,
  'link_flair_text_color': 'dark',
  'upvote_ratio': 0.98,
  'author_flair_background_color': None,
  'ups': 12346,
  'total_awards_received': 27,
  'media_embed': {},
  'thumbnail_width': 140,
  'author_flair_template_id': None,
  'is_original_content': False,
  'user_reports': [],
  'secure_media': None,
  'is_reddit_media_domain': True,
  'is_meta': False,
  'category': N

In [11]:
# Create a list of a subreddits that we want to fetch the top posts for

# There might be a better way to do this, but for right now, I'm just gonna do it manually
# My reasoning for this is that I can grab some less popular reddits that some of the listings
# Right now I have some blatantly right, blatantly left, and apolitical subreddits

# Hopefully I don't introduce any implicit bias into my model lol
subreddits = [
    #Quarantined subreddits
    'TheRedPill',

    #apolitical (I think)
    'BlackPeopleTwitter',
    'WhitePeopleTwitter',
    'politics',
    'law',
    'news',
    'worldnews',
    'Jokes',
    'funny',
    'gaming',
    'Games',
    'technology',
    'tech',
    'technews',
    'gadgets',
    'space',
    'science',
    'nextfuckinglevel',
    'dataisbeautiful',
    'MurderedByWords',
    'mildlyinteresting',
    'Damnthatsinteresting',
    'mildlyinfuriating',
    'wallstreetbets',
    'Superstonk',
    'shitposting',
    'pcmasterrace',
    'teenagers',
    'PublicFreakout',
    'memes',
    'AskReddit',
    'AskAnAmerican',
    'GamingCircleJerk',
    'nottheonion',
    'gatesopencomeonin',
    'wholesomememes',
    'wholesome',
    'interestingasfuck',
    'PoliticalCompassMemes',
    'PoliticalHumor',
    'AskMen',
    'AskWomen',
    'TooAfraidToAsk',
    'MadeMeSmile',
    'rareinsults',
    'UFOs',
    'HighStrangeness',
    'todayilearned',
    'RoastMe',
    'CryptoCurrency',
    'Wellthatsucks',
    'barstoolsports',
    'awfuleverything',
    'unpopularopinion',
    'atheism',
    'Christianity',
    'Catholicism',
    'changemyview',
    'Coronavirus',
    'covid19',
    'coronavirusnyc',
    'Masks4All',
    'consipiratard',
    'environment',
    'environment2',
    'TrollXChromosomes',
    'TrollYChromosome',
    'WikiLeaks',
    '4chan',
    'greentext',
    'NoStupidQuestions',
    'offmychest',
    'texas',
    'TexasPolitics',
    'CanadaPolitics',
    'florida',
    'terriblefacebookmemes',
    'insanepeoplefacebook',
    'ShitMomGroupsSay',
    'masskillers',
    'Parenting',
    'Marriage',
    'neoliberal',
    'spacex',
    'savedyouaclick',
    'socialmedia',
    'moderatepolitics',
    'virginvschad',
    'CoronavirusCircleJerk',
    'OutOfTheLoop',

    #left (I think)
    'democrats',
    'antiwork',
    'Anticonsumption',
    'SelfAwarewolves',
    'niceguys',
    'onejoke',
    'Feminism',
    'WitchesVsPatriarchy',
    'AreTheStraightsOK',
    'ToiletPaperUSA',
    'LeopardsAteMyFace',
    'TwoXChromosomes',
    'Anarchism',
    'TopMindsOfReddit',
    'esist',
    'MarchAgainstNazis',
    'exredpill',
    'MensLib',
    'lgbt',
    'gay',
    'bisexual',
    'AgainstHateSubreddits',
    'traaaaaaannnnnnnnnns',
    'trans',
    'GenderCynical',
    'EnoughTrumpSpam',
    'ABoringDystopia',
    'boringdystopia',
    'pointlesslygendered',
    'Persecutionfetish',
    'TheRightCantMeme',
    'MurderedByAOC',
    'LateStageCapitalism',
    'TheBluePill',
    'justneckbeardthings',
    'NotHowGirlsWork',
    'CapitalismSux',
    'GreenAndPleasant',
    'religiousfruitcake',
    'FragileWhiteRedditor',
    'trumpvirus',
    'socialism',
    'asktransgender',
    'SandersForPresident',
    'MtF',
    'ftm',
    'NonBinary',
    'HermanCainAward',
    'lostgeneration',
    'stupidpol',
    'Qult_Headquarters',
    'IncelTear',
    'ParlerWatch',
    'ShitLiberalsSay',
    'Hasan_Piker',
    'DemocraticSocialism',
    'SapphoAndHerFriend',

    #right (I think)
    'JoeRogan',
    'dankmemes',
    'Memes_Of_The_Dank',
    'okbuddyretard',
    'okmatewanker',
    'pussypassdenied',
    'antifeminists',
    'MensRights',
    'ProudMaleFeminists',
    'WhereAreTheFeminists',
    'FeminismUncensored',
    'SRSsucks',
    'Conservative',
    'JordanPeterson',
    'conspiracy',
    'conspiracy_commons',
    'tucker_carlson',
    'TheTrumpZone',
    'LouderWithCrowder',
    'Libertarian',
    'AskThe_Donald',
    'walkaway',
    'ConservativesOnly',
    'IncelsCircleJerk',
    'BlackPillScience',
    'Blubber_Whaling',
    'memegender',
    'SocialJusticeInAction',
    'FreeSpeech',
    'libsofreddit',
    'EnoughCommieSpam',
    'TheLeftCantMeme',
    'Anarcho_Capitalism',
    'ar15',
    'guns',
    'canadaguns'
    'progun',
    'gunpolitics',
    'Firearms',
    'CCW',
    'Glocks',
    'tacticalgear',
    'liberalgunowners',
    'HillaryForPrison',
    'OffensiveJokes',
    'climateskeptics',
    'LockdownCriticalLeft',
    'FightingFakeNews',
    'Republican',
]

In [12]:
# Now, lets build a dataframe of the top posts of all the subreddits in this list

messy_df = pd.DataFrame()
params = {'limit' : 100} # get the top 100 posts || Worth noting that this will not pull more than 100 posts from a single subreddit

for subreddit in subreddits:
    print(subreddit)

    #top posts of all time
    response = requests.get(f"https://oauth.reddit.com/r/{subreddit}/top/?t=all", headers=headers, params = params)

    #TODO: There is a lot of data that isn't being captured here that could be added
    for post in response.json()['data']['children']:
        messy_df = messy_df.append({
            'created_utc' : post['data']['created_utc'],
            'subreddit' : post['data']['subreddit'],
            'subreddit_subscribers' : post['data']['subreddit_subscribers'],
            'name_code' : post['data']['name'],
            'title' : post['data']['title'],
            'text' : post['data']['selftext'],
            'upvote_ratio' : post['data']['upvote_ratio'],
            'num_comments' : post['data']['num_comments'],
            'url' : post['data']['url'] # this is just a url link to picture media
        }, ignore_index = True)

    response = requests.get(f"https://oauth.reddit.com/r/{subreddit}/top/?t=year", headers=headers, params = params)

    #TODO: There is a lot of data that isn't being captured here that could be added
    for post in response.json()['data']['children']:
        messy_df = messy_df.append({
            'created_utc' : post['data']['created_utc'],
            'subreddit' : post['data']['subreddit'],
            'subreddit_subscribers' : post['data']['subreddit_subscribers'],
            'name_code' : post['data']['name'],
            'title' : post['data']['title'],
            'text' : post['data']['selftext'],
            'upvote_ratio' : post['data']['upvote_ratio'],
            'num_comments' : post['data']['num_comments'],
            'url' : post['data']['url'] # this is just a url link to picture media
        }, ignore_index = True)

TheRedPill
BlackPeopleTwitter
WhitePeopleTwitter
politics
law
news
worldnews
Jokes
funny
gaming
Games
technology
tech
technews
gadgets
space
science
nextfuckinglevel
dataisbeautiful
MurderedByWords
mildlyinteresting
Damnthatsinteresting
mildlyinfuriating
wallstreetbets
Superstonk
shitposting
pcmasterrace
teenagers
PublicFreakout
memes
AskReddit
AskAnAmerican
GamingCircleJerk
nottheonion
gatesopencomeonin
wholesomememes
wholesome
interestingasfuck
PoliticalCompassMemes
PoliticalHumor
AskMen
AskWomen
TooAfraidToAsk
MadeMeSmile
rareinsults
UFOs
HighStrangeness
todayilearned
RoastMe
CryptoCurrency
Wellthatsucks
barstoolsports
awfuleverything
unpopularopinion
atheism
Christianity
Catholicism
changemyview
Coronavirus
covid19
coronavirusnyc
Masks4All
consipiratard
environment
environment2
TrollXChromosomes
TrollYChromosome
WikiLeaks
4chan
greentext
NoStupidQuestions
offmychest
texas
TexasPolitics
CanadaPolitics
florida
terriblefacebookmemes
insanepeoplefacebook
ShitMomGroupsSay
masskillers
Pa

In [13]:
messy_df['subreddit'].value_counts()

TheRedPill              200
FragileWhiteRedditor    200
TheRightCantMeme        200
MurderedByAOC           200
LateStageCapitalism     200
                       ... 
SRSsucks                101
environment2            101
Blubber_Whaling         101
memegender               76
ProudMaleFeminists       39
Name: subreddit, Length: 193, dtype: int64

In [14]:
messy_df = messy_df.drop_duplicates(keep = 'first')

In [15]:
print(f"Total Subreddits: {len(messy_df['subreddit'].value_counts())}")
print(f"Total Posts: {len(messy_df)}")

Total Subreddits: 193
Total Posts: 33045


In [16]:
messy_df['image?'] = messy_df['url'].apply(lambda x: ('jpg' in x) or ('png' in x) or ('jpeg' in x))

In [17]:
messy_df['url'].apply(lambda x: ('gif' in x)).value_counts()

False    32482
True       563
Name: url, dtype: int64

In [18]:
messy_df['image?'].value_counts()

True     18924
False    14121
Name: image?, dtype: int64

In [19]:
messy_df['title'].isna().value_counts()

False    33045
Name: title, dtype: int64

In [20]:
messy_df['text'].isna().value_counts()

False    33045
Name: text, dtype: int64

In [21]:
#Image processing: do on windows machine because tessaract only runs on linux and windows
pytesseract.pytesseract.tesseract_cmd =  r'C:\Program Files\Tesseract-OCR\tesseract'

image_text = []

for index, row in messy_df.iterrows():
    if row['image?']:
        try:
            response = requests.get(row['url'])
            image = Image.open(BytesIO(response.content))
            image_string = pytesseract.image_to_string(image)
        except:
            image_string = ''
    else:
        image_string = ''

    image_text.append(image_string)

c:\Users\agcwi\anaconda3\lib\site-packages\PIL\Image.py:2918: DecompressionBombWarning: Image size (121484484 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\agcwi\anaconda3\lib\site-packages\PIL\Image.py:2918: DecompressionBombWarning: Image size (117331669 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [22]:
messy_df['image_text'] = image_text

In [23]:
pd.to_pickle(messy_df, '../data/messy_df.pkl')